In [ ]:
%load_ext autoreload
%autoreload 2
from scipy.interpolate import BSpline, splev
from multiple_flux_surfaces_class import multiple_flux_surfaces
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from grad_shafranov_class import GS_Solution
import numpy as np
from curve_fitting.curve_fitting_class import curve_fitter

In [ ]:
flux_function = GS_Solution(formfactor="Dshape")
# flux_function = GS_Solution(formfactor="spherical tokamak NSTX")
# flux_function = GS_Solution(formfactor="ITER from Cerfon")
# flux_function = GS_Solution(formfactor="circular shaped tokamak")
curve_fitting_object =curve_fitter(solverClassObject=flux_function)
flux_surfaces = multiple_flux_surfaces(curve_fitting_class_object=curve_fitting_object)

In [ ]:
#getting coefficents of all necessary curves in log space near the axis

m_max_project_beg = 1
m_max_project_end = 15
number_of_s_points = 15
coefficients1 = []
coefficients2 = []
coefficients4 = []
coefficients60 = []
s = np.logspace(-4,0,number_of_s_points)
for m_final in range(m_max_project_beg, m_max_project_end+1):
    coeffs_for_1m_final = []
    coeffs_for_2m_final = []
    coeffs_for_4m_final = []
    coeffs_for_60m_final = []
    for i in range(len(s)):
        coeffs_for_1m_final.append(curve_fitting_object.get_full_contour(s_fs=s[i], m_max_start_first=1, m_max_end_first=m_final, m_max_end=m_final, root_search=True))
        coeffs_for_2m_final.append(curve_fitting_object.get_full_contour(s_fs=s[i], m_max_start_first=1, m_max_end_first=m_final, m_max_end=2*m_final, root_search=True))
        coeffs_for_4m_final.append(curve_fitting_object.get_full_contour(s_fs=s[i], m_max_start_first=1, m_max_end_first=m_final, m_max_end=4*m_final, root_search=True))
        coeffs_for_60m_final.append(curve_fitting_object.get_full_contour(s_fs=s[i], m_max_start_first=1, m_max_end_first=m_final, m_max_end=60, root_search=True))
    
    
    coefficients1.append(coeffs_for_1m_final)
    coefficients2.append(coeffs_for_2m_final)
    coefficients4.append(coeffs_for_4m_final)
    coefficients60.append(coeffs_for_60m_final)
    print(f'm_final:{m_final}')


In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))

for m_final in range(m_max_project_beg, m_max_project_end+1):
    error_curve1 = np.zeros(number_of_s_points)
    error_curve2 = np.zeros(number_of_s_points)
    error_curve4 = np.zeros(number_of_s_points)
    error_curve60 = np.zeros(number_of_s_points)
    for i in range(number_of_s_points):
        x,xmap =  coefficients1[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve1[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
        x,xmap =  coefficients2[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve2[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
        x,xmap =  coefficients4[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve4[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
        x,xmap =  coefficients60[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve60[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
    
    
    ax[0,0].plot(s,error_curve1, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])
    ax[0,1].plot(s,error_curve2, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])
    ax[1,0].plot(s,error_curve4, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])
    ax[1,1].plot(s,error_curve60, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])

f = lambda x: (x.set_xscale('log'), x.set_yscale('log'))
np.vectorize(f)(ax)
ax[0,0].legend(mode='expand',ncol = 5, bbox_to_anchor = (0.1,0.3,2,1))


In [ ]:
#plotting error of previously calculates curves

fig, ax = plt.subplots(figsize = (10, 10))

colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))
for m_final in range(m_max_project_beg, m_max_project_end+1):
    error_curve = np.zeros(number_of_s_points)
    print(f'm = {m_final}')
    for i in range(len(s)):
        x,xmap =  coefficients1[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
    
    ax.plot(s,error_curve, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('s', fontsize=15)
ax.set_ylabel('relative error in s',fontsize=15)
ax.legend(bbox_to_anchor = (1,1))

In [ ]:
coeffs_for_s_equal_1 = []
for m_final in range(m_max_project_beg, m_max_project_end+1):
    coeffs_for_s_equal_1.append(curve_fitting_object.get_full_contour(s_fs=1, m_max_start_first=1, m_max_end_first=m_final, m_max_end=60, root_search=True))




In [ ]:
Rhat_0 = np.zeros(len(coeffs_for_s_equal_1))
Zhat_0 = np.zeros(len(coeffs_for_s_equal_1))
# print(coeffs_for_s_equal_1[0][1])

fig, ax = plt.subplots(figsize = (8, 8))
fig2, ax2 = plt.subplots(figsize = (8, 8))

end =20
colors = plt.cm.jet(np.linspace(0,1,end, endpoint=True))

for j in range(end):
    for i in range(len(coeffs_for_s_equal_1)):
        Coeff, coeffs_map = coeffs_for_s_equal_1[i]
        Coeff = np.asarray(Coeff)
        Rhat_0[i] = Coeff[coeffs_map['str_r_c']+j]
        Zhat_0[i] = Coeff[coeffs_map['str_z_c']+j]
        # Rhat_0 = Rhat_0 / Rhat_0[-1]
        # Zhat_0 = Zhat_0 / Zhat_0[-1]
        Rhat_0 = Rhat_0 
        Zhat_0 = Zhat_0 
        
        
    
      
    ax.plot(np.linspace(1,15,15, endpoint=True),abs(Rhat_0), color=colors[j], label = f'Rhat_{j}')
    ax2.plot(np.linspace(1,15,15, endpoint=True),abs(Zhat_0), color=colors[j], label = f'Zhat_{j}')


ax.set_yscale('log')
ax2.set_yscale('log')
ax.legend()
ax2.legend()
ax.set_xlabel(r'$m_{max}$', fontsize = 15)
ax2.set_xlabel(r'$m_{max}$', fontsize = 15)
ax.set_ylabel('Relative coefficient value', fontsize = 15)
ax2.set_ylabel('Relative coefficient value', fontsize = 15)
ax.legend(bbox_to_anchor = (1,1))
ax2.legend(bbox_to_anchor = (1,1))

# print(Rhat_0)
# print(Zhat_0)
# print(Coeff)

In [ ]:
print(Rhat_0)
plt.plot(abs(Rhat_0))
plt.yscale('log')
plt.xlabel(r'$m_{max}$')
plt.ylabel(r"$\hat{R}$")

In [ ]:
plt.yscale('log')
plt.xlabel(r'$m_{max}$')
plt.ylabel(r"$\hat{Z}$")
plt.plot(abs(Zhat_0))

In [ ]:
m_max_project_beg = 1
m_max_project_end = 15
number_of_s_points = 15

s = np.logspace(-4,-1,number_of_s_points, endpoint=True)
# s = np.linspace(0,1,number_of_s_points,endpoint=True)
# s[0]=1.0e-4

q_data_points = np.zeros((m_max_project_end+1-m_max_project_beg,number_of_s_points))

colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))
for m_final in range(m_max_project_beg, m_max_project_end+1):
    print(f'm = {m_final}')
    for i in range(len(s)):
        q_data_points[m_final - m_max_project_beg][i] = flux_surfaces.safety_factor_with_dedicated_contour(s=s[i],m_max_end=60, m_max_start_first=1, m_max_end_first=m_final)
    


In [ ]:

colors = plt.cm.jet(np.linspace(0,1,number_of_s_points, endpoint=True))
fig, ax = plt.subplots(figsize = (8,8))
plot_y = np.transpose(abs(q_data_points/q_data_points[-1]-1)+1.0e-16)

for i in range(number_of_s_points):
    ax.plot(np.arange(start=m_max_project_beg, stop=m_max_project_end+1),plot_y[i],label =f"s:{round(s[i], 4)}", color=colors[i])
ax.set_xlabel(r'$m_{max}$', fontsize=15)
ax.set_ylabel(r'$|\frac{q(s)_{i}}{q(s)_{m_{max}}}-1| + 10^{-16}$',fontsize=15)
ax.legend()
ax.legend(bbox_to_anchor = (1,1))
ax.set_yscale('log')


In [ ]:
np.arange(5)*np.ones((2,5))

In [ ]:
x ,y = q_data_points.shape
mode_difference = np.zeros((x-1,y))
for i in range(len(q_data_points)-1):
    mode_difference[i] = q_data_points[i+1]-q_data_points[i]

print(mode_difference)

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
ax.plot(np.arange(start=m_max_project_beg, stop=m_max_project_end),np.abs(mode_difference),label = [f"s:{round(s[i], 4)}" for i in range(len(mode_difference.transpose()))])
# ax.set_xscale('log')
ax.set_xlabel('mode')
ax.set_ylabel('q(Safety Factor) difference')
ax.legend(bbox_to_anchor = (1.2,1))
color = plt.cm.jet(np.linspace(0,1,len(mode_difference.transpose())))
print(color)
print(s)

In [ ]:
m_max_project_beg = 1
m_max_project_end = 10
number_of_s_points = 15

s = np.logspace(-4,-1,number_of_s_points, endpoint=True)
fig, ax = plt.subplots(figsize = (8,8))

colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))

previous = np.zeros(number_of_s_points)

for m_final in range(m_max_project_beg, m_max_project_end+1):
    q_curve = np.zeros(number_of_s_points)
    print(f'm = {m_final}')
    for i in range(len(s)):
        q_curve[i] = flux_surfaces.safety_factor_with_dedicated_contour(s=s[i],m_max_end=60, m_max_start_first=1, m_max_end_first=m_final)
    if m_final == m_max_project_beg:
        previous = q_curve
        continue
    
    ax.plot(s,q_curve-previous, label=f'm_final:{m_final} - {m_final-1}',  color = colors[m_final-m_max_project_beg])

ax.set_xscale('log')
ax.set_xlabel('s')
ax.set_ylabel('q(Safety Factor)')
ax.legend(bbox_to_anchor = (1.2,1))